<a href="https://colab.research.google.com/github/chayn1v/temp_lightgbm/blob/main/EDA_for_text_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## plan to change for lvl 3.
A0. Changed grouplist set. A very slight improvement in training: training error is higher and validation is a tiny bit lower. 

A. mean encode text features (instead of keeping them categorical)

B. get rid of days_lag_x -- overfitting

C. Deep learning similarity. For every item find a most similar one(s) on the list. For every item (in shop/month) find the most similar item (in history?)

##Data prep for LightGBM.

1.test set is augmented. It would be a better idea not to merge categorial and shop features in all_data as I did.

2. Maybe it is better to do aggregations from the original sales file than from monthly aggregates?

3.updates: for the targets:  filled 0 after aggregations
4.excude =False
  


In [ ]:
try:
  drive
  print('Drive Mounted already')
except:
  NameError
  from google.colab import drive
  drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
import sys
 
import pandas as pd
import numpy as np
from itertools import product
import pandas_profiling
import gc
from tqdm import notebook  
import matplotlib.pyplot as plt
 

In [ ]:
def downcast_dtypes(df):
    ''' provided for efficiency (in class)
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int16`
    '''
    before=df.memory_usage(index=True).sum()/1024**2
    print('original dataframe size', before,' MBs')
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype in [ "float64","float32"]]
    int_cols =   [c for c in df if df[c].dtype in ["int64","int32"]]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    after=df.memory_usage(index=True).sum()/1024**2
    reduction=after/before
    print('downcasted  dataframe  size', after,' MBs')
    print('fraction of original memory size: ', reduction)
    print(df.dtypes)
    return df    
  
 


In [ ]:
 
path='drive/My Drive/Colab Notebooks/week_3/'
# loading files 
sales = pd.read_csv(path+'sales_train_v2.csv', encoding='utf8')
items = pd.read_csv(path+'items.csv', encoding='utf8')   
shops = pd.read_csv(path+'shops.csv', encoding='utf8')
item_cats=pd.read_csv(path+'item_categories.csv', encoding='utf8')  
test=pd.read_csv(path+'test.csv', encoding='utf8')
#cons_conf_ind=pd.read_csv(path+'cons_conf_ind.csv', encoding='utf8')

#custom processed:
path1='drive/My Drive/'
item_categ_p=pd.read_csv(path1+'item_categ_p.csv', encoding='utf8')
item_categ_p=item_categ_p.rename(columns={"игр_all": "games_all"})
shops2=pd.read_csv(path1+'shops2.csv', encoding='utf8')
shops2=shops2.drop('shop_name',axis=1)
s_items1=pd.read_csv(path1+'s_items1.csv', encoding='utf8')

#downcast
datasets=[sales, items,shops, item_cats, test, item_categ_p,shops2,s_items1 ] 
for df in datasets:
  df=downcast_dtypes(df)

del datasets
gc.collect()  
   
#print(s_items1.head())
#print(s_items1.sparse.density)

s_items1_cols= list(s_items1.columns.values)
print(s_items1_cols)

original dataframe size 134.3926239013672  MBs
downcasted  dataframe  size 50.39731025695801  MBs
fraction of original memory size:  0.3750005676944396
date               object
date_block_num      int16
shop_id             int16
item_id             int16
item_price        float16
item_cnt_day      float16
dtype: object
original dataframe size 0.5075531005859375  MBs
downcasted  dataframe  size 0.25383758544921875  MBs
fraction of original memory size:  0.5001202537353816
item_name           object
item_id              int16
item_category_id     int16
dtype: object
original dataframe size 0.00103759765625  MBs
downcasted  dataframe  size 0.00069427490234375  MBs
fraction of original memory size:  0.6691176470588235
shop_name    object
shop_id       int16
dtype: object
original dataframe size 0.00140380859375  MBs
downcasted  dataframe  size 0.00092315673828125  MBs
fraction of original memory size:  0.657608695652174
item_category_name    object
item_category_id       int16
dtype: obje

In [ ]:
 # output format
pd.set_option('display.max_rows', 150)

pd.set_option('display.max_columns', 150)

In [ ]:
number_of_items_in_sales_per_month=sales.groupby('date_block_num').agg(number_of_items_in_sales =('item_id', 'nunique') )
number_of_items_in_test=test ['item_id'].nunique()
total_number_of_items_in_sales=sales ['item_id'].nunique() 



items_in_test=set(test ['item_id'].unique() )
items_in_sales=set(sales ['item_id'].unique() )
items_in_test_not_in_sales=items_in_test-items_in_sales


print('number_of_items_in_test',number_of_items_in_test)
print('items_in_test_not_in_sales',len(items_in_test_not_in_sales))
#print('number_of_items_in_sales_per_month',number_of_items_in_sales_per_month)
print('number_of_items_in_sales_total',total_number_of_items_in_sales )
print('items_in_test_not_in_sales +number_of_items_in_sales_total',len(items_in_test_not_in_sales) +total_number_of_items_in_sales )
print(items['item_id'].nunique())

number_of_items_in_test 5100
items_in_test_not_in_sales 363
number_of_items_in_sales_total 21807
items_in_test_not_in_sales +number_of_items_in_sales_total 22170
22170


In [ ]:
   
#Editing the Kaggle test set to the same format as sales 
test1=test.drop(['ID'], axis=1)
test1['date' ]='01.11.2016'
test1['date_block_num' ]=34
#dud values not to be used
test1[ 'item_price'  ]=1
test1[  'item_cnt_day']=1
print(test1.head(3))
#required column order: date  date_block_num  shop_id  item_id  item_price  item_cnt_day 

sales=pd.concat([sales,test1], sort=True , ignore_index=True)
print(sales.loc[sales.date_block_num==34].head(3).T)
print(sales.loc[sales.date_block_num==33].head(3).T)

########################################################################
index_cols=['shop_id', 'item_id', 'date_block_num']
sales1=pd.concat([sales,test1], ignore_index=True, sort=False, keys=index_cols )
print(sales1.loc[sales1.date_block_num==34].head(3).T)
print(sales1.loc[sales1.date_block_num==33].head(3).T)


   shop_id  item_id        date  date_block_num  item_price  item_cnt_day
0        5     5037  01.11.2016              34           1             1
1        5     5320  01.11.2016              34           1             1
2        5     5233  01.11.2016              34           1             1
                   2935849     2935850     2935851
date            01.11.2016  01.11.2016  01.11.2016
date_block_num          34          34          34
item_cnt_day             1           1           1
item_id               5037        5320        5233
item_price               1           1           1
shop_id                  5           5           5
                   2882335     2882336     2882337
date            23.10.2015  05.10.2015  02.10.2015
date_block_num          33          33          33
item_cnt_day             1           1           1
item_id              13315       13880       13881
item_price             649         229         659
shop_id                 45          45   

In [ ]:
 items_=items[items.item_id.isin(items_in_test_not_in_sales)]
 

In [ ]:
#custom processing the data, creating features

def sales1_ (sales, exclude=False, only_test_shops=False):
  '''corrects duplicate shops, modifies sales''' 
  #sales1=sales.copy()

  # shop 11 has the same address as shop 10, data exists for only one month and is an error (or alternative name for the same shop) 
  sales.loc[(sales.shop_id==11) &(sales.date_block_num==25), 'shop_id']=10
  #We remove rows where data is reported yearly instead of monthly (in order to create item life features)
   

# Identifying shops with the same address
# 0      !Якутск Орджоникидзе, 56 фран
# 1      !Якутск ТЦ "Центральный" фран
# 57      Якутск Орджоникидзе, 56
# 58      Якутск ТЦ "Центральный"
#This may be considered as one shop (different businesses in 1 moll) NOT IMPLEMENTED
#23                  москва тк буденовский пав а2                 10     0
#24                  москва тк буденовский пав к7                 10     0
  for months in [0,1]:
    for shopp in [ 57,58 ]:
      sales.loc[(sales.shop_id==shopp-57) &(sales.date_block_num==months), 'shop_id']=shopp
  
# Excluding the shops which provided only one month per year data
#This may change the the count of items (some items exclusively belong to this shop))
  if exclude==True:
    sales=sales.loc[(sales.shop_id != 9) &(sales.shop_id != 20)]
     
  
  if only_test_shops==True:  #Default is false
    sales=sales.loc[(sales.shop_id != 9) &(sales.shop_id != 20)]
    sales=sales.loc[sales.shop_id.isin(test_shops) ]
    test_shops=np.sort(test.drop_duplicates(subset='shop_id').shop_id.values)
    test_shops=list(test_shops)
    print(test_shops )
    print(len(test_shops) )
  
  return sales 

 

#clipping and transforming price 
sales.item_price=sales.item_price.clip(1,100000)
#sales.item_price=np.log( sales.item_price+1)


In [ ]:
sales=sales1_ (sales,exclude=False)

#clipping and transforming price 
sales.item_price=sales.item_price.clip(1,100000)
#sales.item_price=np.log( sales.item_price+1)


print(test.head)

<bound method NDFrame.head of            ID  shop_id  item_id
0           0        5     5037
1           1        5     5320
2           2        5     5233
3           3        5     5232
4           4        5     5268
...       ...      ...      ...
214195  17587       45    18454
214196  17588       45    16188
214197  17589       45    15757
214198  17590       45    19648
214199  17591       45      969

[214200 rows x 3 columns]>


In [ ]:
#An example of shop which just opened and has no significant sales this month

print(sales.loc[(sales.shop_id==34)&(sales.date_block_num==18)])
#x=sales.loc[(sales.shop_id==34)&(sales.date_block_num==18)].index 
#sales=sales.drop(x) 

               date  date_block_num  item_cnt_day  item_id  item_price  \
1803319  31.07.2014              18           1.0    20949         5.0   
1803320  31.07.2014              18           1.0    11030       149.0   

         shop_id  
1803319       34  
1803320       34  


In [ ]:
#Locating all shop/months with less than 20 sales
gb=sales.groupby(['shop_id','date_block_num']).agg(target_shop=("item_cnt_day","sum")).target_shop.sort_values()

gb=gb[gb.le(20)]
print(gb[gb.le(20)] )
print(gb[gb.le(20)].index   )



shop_id  date_block_num
33       27               -1.0
27       32               -1.0
8        3                -1.0
9        27               -1.0
34       18                2.0
Name: target_shop, dtype: float64
MultiIndex([(33, 27),
            (27, 32),
            ( 8,  3),
            ( 9, 27),
            (34, 18)],
           names=['shop_id', 'date_block_num'])


In [ ]:
for (shop,month) in list(gb[gb.le(20)].index.values):
   ind=sales.loc[(sales.shop_id==shop)&(sales.date_block_num==month)].index 
   print(sales.loc[(sales.shop_id==shop)&(sales.date_block_num==month)])
   sales=sales.drop(ind)


               date  date_block_num  item_cnt_day  item_id  item_price  \
2561462  24.04.2015              27          -1.0     6673      7992.0   

         shop_id  
2561462       33  
               date  date_block_num  item_cnt_day  item_id  item_price  \
2878882  30.09.2015              32          -1.0     9250      1300.0   

         shop_id  
2878882       27  
              date  date_block_num  item_cnt_day  item_id  item_price  shop_id
366475  01.04.2013               3          -1.0     4809       599.0        8
               date  date_block_num  item_cnt_day  item_id  item_price  \
2600214  22.04.2015              27          -1.0     5672      3190.0   

         shop_id  
2600214        9  
               date  date_block_num  item_cnt_day  item_id  item_price  \
1803319  31.07.2014              18           1.0    20949         5.0   
1803320  31.07.2014              18           1.0    11030       149.0   

         shop_id  
1803319       34  
1803320       34  


In [ ]:
sales.date.describe()
sales.date=pd.to_datetime(sales.date,dayfirst = True)
sales.date.describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until


count                 3150043
unique                   1035
top       2016-11-01 00:00:00
freq                   214200
first     2013-01-01 00:00:00
last      2016-11-01 00:00:00
Name: date, dtype: object

In [ ]:
print(sales.columns.values)
#sales.date=pd.to_datetime(sales.date,dayfirst=True)
#print(sales.date.describe())

#history=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num','date']].groupby('item_id').agg(item_start=('date_block_num', 'min'),item_start_date=('date', 'min') , item_end=('date_block_num', 'max'))
history=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num','date']].groupby('item_id').agg(item_start=('date_block_num', 'min') , item_end=('date_block_num', 'max'))
history_shop=sales.loc[sales.item_cnt_day>0][['item_id','date_block_num','shop_id','date']].groupby(['item_id','shop_id']).agg(item_shop_start=('date_block_num', 'min') , item_shop_end=('date_block_num', 'max'))
print(history.head())
print(history_shop.head())

['date' 'date_block_num' 'item_cnt_day' 'item_id' 'item_price' 'shop_id']
         item_start  item_end
item_id                      
0                20        20
1                15        21
2                19        22
3                18        19
4                20        20
                 item_shop_start  item_shop_end
item_id shop_id                                
0       54                    20             20
1       55                    15             21
2       54                    19             22
3       54                    18             19
4       54                    20             20


In [ ]:
 

#sales=sales.copy()
#process date feature Do 1 time!
sales['date']=pd.to_datetime( sales['date'], format="%d.%m.%Y" ) 
#Month feature [0,11] 
sales['month']=sales['date'].dt.month-1 
sales['day_of_mnth']=sales.date.dt.day 
sales=sales.drop('date', axis=1)

print(sales.dtypes)
print(shops.dtypes)
 
sales=pd.merge(sales, items[['item_category_id','item_id'] ], how='left', on='item_id'  ) 


sales = pd.merge(sales, shops2, how='left', on='shop_id') 
sales = pd.merge(sales, item_categ_p, how='left', on='item_category_id')
sales = pd.merge(sales, s_items1, how='left', on='item_id')

item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

print(items.shape)
print(item_category_mapping.shape)
print(sales[2:5].T)

date_block_num      int64
item_cnt_day      float64
item_id             int16
item_price        float64
shop_id             int16
month               int64
day_of_mnth         int64
dtype: object
shop_name    object
shop_id       int16
dtype: object
(22170, 3)
(22170, 2)
                       2       3       4
date_block_num       0.0     0.0     0.0
item_cnt_day        -1.0     1.0     1.0
item_id           2552.0  2554.0  2555.0
item_price         899.0  1709.0  1099.0
shop_id             25.0    25.0    25.0
month                0.0     0.0     0.0
day_of_mnth          5.0     6.0    15.0
item_category_id    58.0    58.0    56.0
shop_type            2.0     2.0     2.0
city_code           11.0    11.0    11.0
games_all            0.0     0.0     0.0
class1               0.0     0.0     0.0
class2               0.0     0.0     0.0
class4               3.0     3.0     3.0
cd_count             0.0     0.0     2.0
bd_count             0.0     0.0     0.0
dvd_count            0.0     0.

In [ ]:
extra_aggregates=True
#lags_for_extras=False
 
if extra_aggregates==True:
  print( item_categ_p.columns.values)
  #
  print(shops2.columns.values)
  new_columns=list((set(item_categ_p.columns.values) | set( shops2.columns.values))-set(['item_category_id' ,'shop_id']))
  print(new_columns)

['item_category_id' 'games_all' 'class1' 'class2' 'class4']
['shop_id' 'shop_type' 'city_code']
['class4', 'city_code', 'games_all', 'class2', 'class1', 'shop_type']


In [ ]:
print(sales.shape) 

#items_in_test=set(test ['item_id'].unique() )
items_in_sales_and_test1=set(sales ['item_id'].unique() )
#items_in_test_not_in_sales=items_in_test-items_in_sales
print('items in sales after modification +test_items',len(items_in_sales_and_test1))



items_in_test=set(test ['item_id'].unique() )
items_in_sales=set(sales ['item_id'].unique() )
items_in_test_not_in_sales=items_in_test-items_in_sales


items_not_present_in_modified_sales_and_test=items_in_test- items_in_sales_and_test1


print('items_not_present_in_modified_sales_and_test',len(items_not_present_in_modified_sales_and_test))

sales.head  
##print(sales.ix[ 17])
print(sales.loc[(sales.item_id==2574)])#&(sales.date_block_num)==0]
#sales=sales.rename(columns={"игр_all": "games_all"})
sales=downcast_dtypes(sales)







(3150043, 41)
items in sales after modification +test_items 22170
items_not_present_in_modified_sales_and_test 0
         date_block_num  item_cnt_day  item_id  item_price  shop_id  month  \
10                    0           2.0     2574       399.0       25      0   
11                    0           1.0     2574       399.0       25      0   
12                    0           1.0     2574       399.0       25      0   
13                    0           2.0     2574       399.0       25      0   
14                    0           1.0     2574       399.0       25      0   
...                 ...           ...      ...         ...      ...    ...   
3124963              34           1.0     2574         1.0       46     10   
3130063              34           1.0     2574         1.0       41     10   
3135163              34           1.0     2574         1.0       44     10   
3140263              34           1.0     2574         1.0       39     10   
3145363              34      

In [ ]:
gc.collect()

14

In [ ]:
'''def grid_df(source_df):
  # by 'date_block_num', for all  pairs ['shop_id', 'item_id'] in sals_df[date_block_num']
  index_cols=['shop_id', 'item_id', 'date_block_num']
  grid = [] 
  
  for block_num in source_df['date_block_num'].unique():
    cur_shops = source_df.loc[source_df['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = source_df.loc[source_df['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items,   [block_num]])),dtype='int16'))

  # Turn the grid into a dataframe
  grid_df = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int16)
  return grid_df
'''
#Generalized grid:
def grid_df1(source_df,index_cols_):
  # by 'date_block_num', for all  other columns in (such as ['shop_id', 'item_id'])  in source_df[date_block_num']
  #index_cols=['shop_id', 'item_id', 'date_block_num']
  cols=index_cols_.copy()
  #print('cols = ', cols)
  cols.remove('date_block_num')
  #print('cols = ', cols)
  grid = [] 
  
  for block_num in source_df['date_block_num'].unique():
    prep_vals=[]
    for col in cols:
      cur_col = source_df.loc[source_df['date_block_num'] == block_num, col].unique()
      prep_vals.append(cur_col)
    prep_vals.append([block_num])  
    grid.append(np.array(list(product(*prep_vals)),dtype='int16'))

  # Turn the grid into a dataframe
  grid_df = pd.DataFrame(np.vstack(grid), columns = index_cols_,dtype=np.int16)
  return grid_df


In [ ]:
def group_by_merge(groupby_cols, source_df, grid, aggregation,fill_zero=True):
  #for iterative merges to grid
   
  agg_0=aggregation[0]
  agg_1=aggregation[1]
  #print(source_df.head())
  
  gb = source_df.groupby(groupby_cols  ).agg(x=agg_1)
  #print(gb.head( ))
   
  db_merged = pd.merge(grid, gb, how='left', on=groupby_cols)
  del gb
  if fill_zero:
    db_merged.x=db_merged.x.fillna(0)
   
  #print(agg_0)
  grid=db_merged.rename(columns={"x": agg_0})
  gc.collect()
  return grid

 



# fooname == 'foo'

def save_csv(df):
  df=downcast_dtypes(df)
  N=df.shape[0]//2
  print(df[N:N+10])
   
  df.to_csv(path+df.name+'.csv', encoding='utf8',index=False)
   

In [ ]:
def create_lags(df,index_cols, shift_range=[1,2,3,4,12], no_lags=[]):
  #index_cols=index_cols+['item_category_id'] should be included
  #index_cols=index_cols+['date_block_num'] should be included
 
  print("index_cols = ", index_cols)
  cols_to_rename = list(set(df.columns.difference(index_cols))-set(no_lags)) 
  print(cols_to_rename)
  for month_shift in notebook.tqdm(shift_range):
    train_shift = df[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    df = pd.merge(df, train_shift, on=index_cols, how='left')#.fillna(0)
  
  if 'target' in set(cols_to_rename):
    cols_to_rename.remove('target')
  if 'days' in set(cols_to_rename):
    cols_to_rename.remove('days')  
  df=df.drop(cols_to_rename,axis=1)
  print('cols_to_rename (dropped)', cols_to_rename)
  return df 
  
def diff(new,old,eps=.1):
  return 2*(new-old)/(new+old+eps)

#cols_diff=['target','mean_price_month','std_price_month', 'target_shop','days','target_item' , 'shop_item_cat_target', 'shop_item_cat_target_per_day']
def difference(df,cols_diff):
  #creates differences of columns with lags 1 and 2
  for col in cols_diff:
    u='{}_diff'.format(col) 
    v='{}_lag_1'.format(col)
    w='{}_lag_2'.format(col)
    df[u]=diff(df[v],df[w])
  return df  
  
def is_nan_check(df,cols=None):
  if cols==None:
    cols=list(df.columns.values)

  for col in cols:
     
    print(col, ' is nan count ', pd.isnull(df[col]).sum(), ' ; mean ',pd.isnull(df[col]).mean())  

In [ ]:
#Executable

index_cols = ['shop_id', 'item_id', 'date_block_num']


grid1=grid_df1( sales,index_cols)
 
#grid1.name="grid1"



In [ ]:
print(grid1.head())
#print(sales.head())
 

   shop_id  item_id  date_block_num
0       59    22154               0
1       59     2552               0
2       59     2554               0
3       59     2555               0
4       59     2564               0


In [ ]:
groupby_cols=['shop_id', 'item_id', 'date_block_num']
arg=["target",('item_cnt_day','sum')]

#res=group_by_merge(groupby_cols, source_df=sales, grid=grid,aggregations= {target:('item_cnt_day','sum')})
 

#gb = sales.groupby(index_cols  ).agg( exec("target=('item_cnt_day','sum')") )# THERE SHOULD NOT BE A FILLNA HERE!  
#db_merged = pd.merge(res, gb, how='left', on=index_cols)
arg_list=[["target",('item_cnt_day','sum')],
          ["mean_price_month",('item_price','mean')],
          ["min_price_month",('item_price','min')],
          ["max_price_month",( 'item_price','max')],
           ["std_price_month",('item_price', 'std')]
          ]

for arg in arg_list : 
  grid1=group_by_merge(groupby_cols, source_df=sales, grid=grid1,aggregation= arg,fill_zero=False)        

In [ ]:
grid1.target=grid1.target.fillna(0)
grid1.target=grid1.target.clip(0,20)

In [ ]:
grid1.loc[grid1['date_block_num']==33].shape 
print(grid1.loc[grid1['date_block_num']==33].shop_id.value_counts())
print(grid1.loc[grid1['date_block_num']==33].shop_id.nunique())

59    5413
58    5413
26    5413
25    5413
24    5413
22    5413
21    5413
20    5413
19    5413
18    5413
16    5413
15    5413
14    5413
12    5413
10    5413
9     5413
7     5413
6     5413
5     5413
4     5413
3     5413
28    5413
31    5413
34    5413
47    5413
57    5413
56    5413
55    5413
53    5413
52    5413
50    5413
49    5413
48    5413
46    5413
35    5413
45    5413
44    5413
42    5413
41    5413
39    5413
38    5413
37    5413
36    5413
2     5413
Name: shop_id, dtype: int64
44


In [ ]:
print(grid1.head(10))
is_nan_check(grid1)
#save_csv(grid1)
#Check
#grid1.groupby(index_cols).target.nunique().head(50)

   shop_id  item_id  date_block_num  target  mean_price_month  \
0       59    22154               0     1.0             999.0   
1       59     2552               0     0.0               NaN   
2       59     2554               0     0.0               NaN   
3       59     2555               0     0.0               NaN   
4       59     2564               0     0.0               NaN   
5       59     2565               0     0.0               NaN   
6       59     2572               0     0.0               NaN   
7       59     2573               0     0.0               NaN   
8       59     2574               0     2.0             399.0   
9       59     2593               0     0.0               NaN   

   min_price_month  max_price_month  std_price_month  
0            999.0            999.0              NaN  
1              NaN              NaN              NaN  
2              NaN              NaN              NaN  
3              NaN              NaN              NaN  
4        

In [ ]:
# mean encoding shop/month

index_cols=['shop_id', 'date_block_num']
grid_shop_month=grid_df1( sales,index_cols)


#from grid1
arg=["target_shop",('target','sum')]
grid_shop_month=group_by_merge(groupby_cols=index_cols, source_df=grid1.loc[grid1.target>0], grid=grid_shop_month,aggregation= arg)


#from sales
arg=["days",('day_of_mnth', 'nunique')]
grid_shop_month=group_by_merge(groupby_cols=index_cols, source_df=sales, grid=grid_shop_month,aggregation= arg)

grid_shop_month=grid_shop_month.fillna(0)
   

In [ ]:
#print(grid_shop_month)


#x=grid_shop_month.loc[grid_shop_month.date_block_num==33]
#print(x)
#all are open 31 days  except for 4:   #36 just opened, #12 has unusually less days:
#shops 15 and 24 are closed for a day (have 30 days)?


y36=grid_shop_month.loc[(grid_shop_month.date_block_num==33)&(grid_shop_month.shop_id==36)]
print(y36)
y12=grid_shop_month.loc[(grid_shop_month.date_block_num==32)&(grid_shop_month.shop_id==12)]
print(y12)
y12=grid_shop_month.loc[(grid_shop_month.date_block_num==31)&(grid_shop_month.shop_id==12)]
print(y12)

y15_24=grid_shop_month.loc[(grid_shop_month.date_block_num==33)&(grid_shop_month.shop_id.isin([15,24]))]
print(y15_24)



#print(x.groupby('shop_id').agg(('days','value_counts') ) )
#print(grid_shop_month.loc[grid_shop_month.date_block_num.isin([32,33])].groupby('shop_id').days)

      shop_id  date_block_num  target_shop  days
1548       36              33        330.0    14
      shop_id  date_block_num  target_shop  days
1529       12              32       1671.0    30
      shop_id  date_block_num  target_shop  days
1478       12              31       1321.0    30
      shop_id  date_block_num  target_shop  days
1562       15              33       1223.0    30
1579       24              33       1207.0    30


In [ ]:
days_shop_12=sales.loc[(sales.date_block_num==33)&(sales.shop_id==12)].groupby('day_of_mnth').agg(t=('item_cnt_day',"sum"))
print(days_shop_12)  

print(shops.loc[shops.shop_id==12])

print(sales.loc[(sales.shop_id==12)&(sales.date_block_num==33)&(sales.day_of_mnth==28)])
print(items[items.item_id==20949]       )
print(items[items.item_id==  11373     ]       )

days_shop_15=sales.loc[(sales.date_block_num==33)&(sales.shop_id==15)].groupby('day_of_mnth').agg(t=('item_cnt_day',"sum"))
print(days_shop_15)  




                  t
day_of_mnth        
1              89.0
2              62.0
5              75.0
6              36.0
7              89.0
8              84.0
9              33.0
12             49.0
13             89.0
14            603.0
15             84.0
16            100.0
18              1.0
19             68.0
20             77.0
21             86.0
22             58.0
23             45.0
25              1.0
26             65.0
27             42.0
28           2258.0
29             45.0
30             41.0
              shop_name  shop_id
12  Интернет-магазин ЧС       12
         date_block_num  item_cnt_day  item_id  item_price  shop_id  month  \
2909322              33           1.0    20629     2400.00       12      9   
2909334              33           1.0    19971     3200.00       12      9   
2909348              33           1.0    20195      720.00       12      9   
2909393              33           1.0    20866      699.00       12      9   
2909399              33 

In [ ]:
days_shop_36=sales.loc[(sales.date_block_num==33)&(sales.shop_id==36)].groupby('day_of_mnth').agg(t=('item_cnt_day',"sum"))
print(days_shop_36)

                t
day_of_mnth      
18           37.0
19            9.0
20           16.0
21           13.0
22           25.0
23           18.0
24           33.0
25           20.0
26           11.0
27           34.0
28           15.0
29           17.0
30           34.0
31           48.0


In [ ]:
#fixing days for the test month:
grid_shop_month.loc[grid_shop_month.date_block_num==34,'days']=30
#except shop N12:
grid_shop_month.loc[(grid_shop_month.date_block_num==34)&(grid_shop_month.shop_id==12),'days'] =24
print(grid_shop_month.loc[(grid_shop_month.date_block_num==34)&(grid_shop_month.shop_id==12)] ) 
print(grid_shop_month[grid_shop_month.date_block_num==34].head())

#Target_shop for test month will not be used!

      shop_id  date_block_num  target_shop  days
1588       12              34       5100.0    24
      shop_id  date_block_num  target_shop  days
1581        5              34       5100.0    30
1582        4              34       5100.0    30
1583        6              34       5100.0    30
1584        3              34       5100.0    30
1585        2              34       5100.0    30


In [ ]:
#no_lag=index_cols+days
grid_shop_month=create_lags( grid_shop_month,index_cols, shift_range=[1,2,3,4,12])
grid_shop_month=grid_shop_month.fillna(0)
 
grid_shop_month['month']=grid_shop_month.date_block_num%12
grid_shop_month['year']=grid_shop_month.date_block_num//12


gc.collect()




index_cols =  ['shop_id', 'date_block_num']
['days', 'target_shop']



cols_to_rename (dropped) ['target_shop']


4

In [ ]:
cols_diff=['target_shop']  
grid_shop_month=difference(grid_shop_month,cols_diff)
is_nan_check(grid_shop_month)

shop_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
days  is nan count  0  ; mean  0.0
days_lag_1  is nan count  0  ; mean  0.0
target_shop_lag_1  is nan count  0  ; mean  0.0
days_lag_2  is nan count  0  ; mean  0.0
target_shop_lag_2  is nan count  0  ; mean  0.0
days_lag_3  is nan count  0  ; mean  0.0
target_shop_lag_3  is nan count  0  ; mean  0.0
days_lag_4  is nan count  0  ; mean  0.0
target_shop_lag_4  is nan count  0  ; mean  0.0
days_lag_12  is nan count  0  ; mean  0.0
target_shop_lag_12  is nan count  0  ; mean  0.0
month  is nan count  0  ; mean  0.0
year  is nan count  0  ; mean  0.0
target_shop_diff  is nan count  0  ; mean  0.0


In [ ]:
grid_shop_month.name="grid_shop_month"
save_csv(grid_shop_month)

#It is used and deleted later!


#grid_shop_month=downcast_dtypes(grid_shop_month)
#print(grid_shop_month.head())
#del grid_shop_month

original dataframe size 0.11763381958007812  MBs
downcasted  dataframe  size 0.06191253662109375  MBs
fraction of original memory size:  0.5263157894736842
shop_id                 int16
date_block_num          int16
days                    int16
days_lag_1            float16
target_shop_lag_1     float16
days_lag_2            float16
target_shop_lag_2     float16
days_lag_3            float16
target_shop_lag_3     float16
days_lag_4            float16
target_shop_lag_4     float16
days_lag_12           float16
target_shop_lag_12    float16
month                   int16
year                    int16
target_shop_diff      float16
dtype: object
     shop_id  date_block_num  days  days_lag_1  target_shop_lag_1  days_lag_2  \
811       54              17    30        31.0             4992.0        30.0   
812       53              17    30        31.0             1605.0        30.0   
813       50              17    30        31.0             1830.0        30.0   
814       51              

In [ ]:
#item_month_aggs
index_cols=['item_id', 'date_block_num'] 
grid_item_month=grid_df1( sales,index_cols)

 
args=[["target_item",('target' , 'sum' )],["item_active_shop_count",('shop_id', 'nunique')]]
for arg in args:
  grid_item_month=group_by_merge(groupby_cols=index_cols, source_df=grid1.loc[grid1.target>0], grid=grid_item_month,aggregation= arg)

#grid_item_month["target_item"]=grid_item_month.target_item.fillna(0)
#grid_item_month['item_active_shop_count']=grid_item_month.item_active_shop_count.fillna(0)


grid_item_month['mean_item_per_active_shop']=grid_item_month.target_item/grid_item_month.item_active_shop_count 
#grid_item_month['mean_item_per_active_shop']=grid_item_month['mean_item_per_active_shop'].fillna(0)

grid_item_month=grid_item_month.fillna(0)

In [ ]:
grid_item_month=create_lags( grid_item_month,index_cols, shift_range=[1,2,3,4,12])

grid_item_month=pd.merge(grid_item_month,history, how='left', on='item_id')

#history for each item
#we assume that  the new items in the test set appear because they were sold in one of the shops  so value 0 means 'sold this month for the first time somewhere'
grid_item_month=grid_item_month.fillna(0)

grid_item_month['history']=(grid_item_month['date_block_num']-grid_item_month['item_start' ]).clip(-1,35)
grid_item_month=grid_item_month.drop(['item_end','item_start'],axis=1)
is_nan_check(grid_item_month)

index_cols =  ['item_id', 'date_block_num']
['mean_item_per_active_shop', 'target_item', 'item_active_shop_count']



cols_to_rename (dropped) ['mean_item_per_active_shop', 'target_item', 'item_active_shop_count']
item_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_1  is nan count  0  ; mean  0.0
target_item_lag_1  is nan count  0  ; mean  0.0
item_active_shop_count_lag_1  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_2  is nan count  0  ; mean  0.0
target_item_lag_2  is nan count  0  ; mean  0.0
item_active_shop_count_lag_2  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_3  is nan count  0  ; mean  0.0
target_item_lag_3  is nan count  0  ; mean  0.0
item_active_shop_count_lag_3  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_4  is nan count  0  ; mean  0.0
target_item_lag_4  is nan count  0  ; mean  0.0
item_active_shop_count_lag_4  is nan count  0  ; mean  0.0
mean_item_per_active_shop_lag_12  is nan count  0  ; mean  0.0
target_item_lag_12  is nan count  0  ; mean  0.0
item_active_shop_count_lag_12  i

In [ ]:

print(history)
print(grid_item_month)

         item_start  item_end
item_id                      
0                20        20
1                15        21
2                19        22
3                18        19
4                20        20
...             ...       ...
22165             7        16
22166             9        34
22167             8        34
22168             0         8
22169            14        14

[22167 rows x 2 columns]
        item_id  date_block_num  mean_item_per_active_shop_lag_1  \
0         22154               0                              0.0   
1          2552               0                              0.0   
2          2554               0                              0.0   
3          2555               0                              0.0   
4          2564               0                              0.0   
...         ...             ...                              ...   
239006    18454              34                              1.0   
239007    16188              34         

In [ ]:
cols_diff=["target_item"]  
grid_item_month=difference(grid_item_month,cols_diff)
 

In [ ]:
is_nan_check(grid_item_month,cols=['target_item_diff'])

target_item_diff  is nan count  0  ; mean  0.0


In [ ]:
#print(grid_item_month.head(10))
grid_item_month.name="grid_item_month"
save_csv(grid_item_month)
#print(grid_item_month.head(10))

del grid_item_month
gc.collect()


#print(items)
#print(sales.head())

original dataframe size 25.529129028320312  MBs
downcasted  dataframe  size 10.485177993774414  MBs
fraction of original memory size:  0.4107142857142857
item_id                               int16
date_block_num                        int16
mean_item_per_active_shop_lag_1     float16
target_item_lag_1                   float16
item_active_shop_count_lag_1        float16
mean_item_per_active_shop_lag_2     float16
target_item_lag_2                   float16
item_active_shop_count_lag_2        float16
mean_item_per_active_shop_lag_3     float16
target_item_lag_3                   float16
item_active_shop_count_lag_3        float16
mean_item_per_active_shop_lag_4     float16
target_item_lag_4                   float16
item_active_shop_count_lag_4        float16
mean_item_per_active_shop_lag_12    float16
target_item_lag_12                  float16
item_active_shop_count_lag_12       float16
history                             float16
target_item_diff                    float16
dtype: obj

4

In [ ]:
#shop/month/category
#need to merge item_categories to grid1:
grid1=pd.merge(grid1, items[['item_id','item_category_id']], how='left', on='item_id')
#print(grid1.head(10))


In [ ]:

index_cols=[ 'shop_id', 'item_category_id','date_block_num' ]
grid_item_cat_month=grid_df1( sales,index_cols)

#print(grid_item_cat_month.shape )
#print(grid_item_cat_month.loc[grid_item_cat_month.shop_id==54])
arg_list=[["shop_item_cat_target",('target','sum')],
          ["shop_item_cat_mean_price",('mean_price_month','mean')]]

for arg in arg_list : 
  grid_item_cat_month=group_by_merge(groupby_cols=index_cols, source_df=grid1, grid=grid_item_cat_month,aggregation= arg ,fill_zero=False)

grid_item_cat_month.shop_item_cat_target=grid_item_cat_month.shop_item_cat_target.fillna(0)


to_merge=['shop_id','date_block_num']+['days']
grid_item_cat_month=pd.merge(grid_item_cat_month,grid_shop_month[to_merge],how="left",on=['shop_id','date_block_num'])
#This aggregate is to be done later (when the whole file is assembled)
grid_item_cat_month['shop_item_cat_target_per_day']=grid_item_cat_month.shop_item_cat_target/(grid_item_cat_month.days )
grid_item_cat_month['shop_item_cat_target_per_day']=grid_item_cat_month['shop_item_cat_target_per_day'].fillna(0)  

grid_item_cat_month=grid_item_cat_month.drop([  "days"],axis=1)

grid_item_cat_month=grid_item_cat_month.fillna(0)

In [ ]:
#print(grid1.head())
print(grid_item_cat_month.head())

   shop_id  item_category_id  date_block_num  shop_item_cat_target  \
0       59                37               0                  60.0   
1       59                58               0                   0.0   
2       59                56               0                   1.0   
3       59                59               0                   2.0   
4       59                55               0                 169.0   

   shop_item_cat_mean_price  shop_item_cat_target_per_day  
0                     454.0                      2.000000  
1                       0.0                      0.000000  
2                    1299.0                      0.033333  
3                    1274.0                      0.066667  
4                     271.5                      5.633333  


In [ ]:
grid_item_cat_month=create_lags( grid_item_cat_month,index_cols, shift_range=[1,2,3,4,12])
grid_item_cat_month=grid_item_cat_month.fillna(0)

cols_diff=[ 'shop_item_cat_target','shop_item_cat_target_per_day']  
grid_item_cat_month=difference(grid_item_cat_month,cols_diff)



is_nan_check(grid_item_cat_month)

index_cols =  ['shop_id', 'item_category_id', 'date_block_num']
['shop_item_cat_target_per_day', 'shop_item_cat_target', 'shop_item_cat_mean_price']



cols_to_rename (dropped) ['shop_item_cat_target_per_day', 'shop_item_cat_target', 'shop_item_cat_mean_price']
shop_id  is nan count  0  ; mean  0.0
item_category_id  is nan count  0  ; mean  0.0
date_block_num  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_1  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_2  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_3  is nan count  0  ; mean  0.0
shop_item_cat_target_per_day_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_target_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_mean_price_lag_4  is nan count  0  ; mean  0.0
shop_item_cat_target_per_da

In [ ]:
#N=grid_item_cat_month.shape[0]//2
#print( grid_item_cat_month[N:N+10])
grid_item_cat_month.name="grid_item_cat_month"
save_csv(grid_item_cat_month)
#print(grid_item_cat_month.head(10))

del grid_item_cat_month,grid_shop_month
gc.collect()
 

original dataframe size 6.05572509765625  MBs
downcasted  dataframe  size 4.5417938232421875  MBs
fraction of original memory size:  0.75
shop_id                                  int16
item_category_id                         int16
date_block_num                           int16
shop_item_cat_target_per_day_lag_1     float16
shop_item_cat_target_lag_1             float16
shop_item_cat_mean_price_lag_1         float16
shop_item_cat_target_per_day_lag_2     float16
shop_item_cat_target_lag_2             float16
shop_item_cat_mean_price_lag_2         float16
shop_item_cat_target_per_day_lag_3     float16
shop_item_cat_target_lag_3             float16
shop_item_cat_mean_price_lag_3         float16
shop_item_cat_target_per_day_lag_4     float16
shop_item_cat_target_lag_4             float16
shop_item_cat_mean_price_lag_4         float16
shop_item_cat_target_per_day_lag_12    float16
shop_item_cat_target_lag_12            float16
shop_item_cat_mean_price_lag_12        float16
shop_item_cat_ta

69

In [ ]:
s_items1_cols=s_items1.columns.values
print(s_items1_cols) 
s_items1_cols=[x for x in s_items1_cols if "text" in x]+['item_id']
 
print(s_items1_cols) 
#need to merge item text features to grid1:
#grid1=pd.merge(grid1, s_items1[s_items1_cols], how='left', on='item_id')
grid1=pd.merge(grid1, s_items1, how='left', on='item_id')
grid1=pd.merge(grid1,item_categ_p, how='left', on='item_category_id')

grid1=pd.merge(grid1,shops2, how='left', on='shop_id')

 
 


print(grid1.head(10))


  
  

['item_id' 'cd_count' 'bd_count' 'dvd_count' 'lp_count' 'code_item_id'
 'code_item_id1' 'text_feature_0' 'text_feature_1' 'text_feature_2'
 'text_feature_3' 'text_feature_4' 'text_feature_5' 'text_feature_6'
 'text_feature_7' 'text_feature_8' 'text_feature_9' 'text_feature_10'
 'text_feature_11' 'text_feature_12' 'text_feature_13' 'text_feature_14'
 'text_feature_15' 'text_feature_16' 'text_feature_17' 'text_feature_18'
 'text_feature_19' 'text_feature_20']
['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'item_id']
   shop_id  item_id  date_block_num  target  mean_price_month  \
0       59    22154               0     1.0             999.0   
1      

In [ ]:
gc.collect() 
print(shops2.columns.values)
print(item_categ_p.columns.values)
print( new_columns )


['shop_id' 'shop_type' 'city_code']
['item_category_id' 'games_all' 'class1' 'class2' 'class4']
['class4', 'city_code', 'games_all', 'class2', 'class1', 'shop_type']


In [ ]:
#text_features/month
s_items1_cols=s_items1.columns.values
s_items1_cols=[x for x in s_items1_cols if "text" in x]
print(s_items1_cols) 
print(type(new_columns),new_columns )
#grouplist=['shop_id']+s_items1_cols+[ 'date_block_num']  
grouplist=s_items1_cols+[ 'date_block_num']  
     
print(grouplist)

#Cannot use Grid function due to memory limitations
#grid_grouplist=grid_df1( sales,grouplist) 
 
 

  
 




['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20']
<class 'list'> ['class4', 'city_code', 'games_all', 'class2', 'class1', 'shop_type']
['text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'date_block_num']


In [ ]:
#Due to out of memory error we cannot use groupby_merge

#there is an error before save: item_id should not be there!!!

gb = grid1.loc[(grid1.target>0)&((grid1[s_items1_cols]>0).any(axis=1))].groupby(grouplist).agg(item_groupllist_target=('target' , 'sum' ),item_groupllist_count=('item_id','nunique'))

grid_grouplist = pd.merge(grid1, gb, how='left', on=grouplist)

grid_grouplist.item_groupllist_target=grid_grouplist.item_groupllist_target.fillna(0)
grid_grouplist.item_groupllist_count=grid_grouplist.item_groupllist_count.fillna(0)

grid_grouplist['ratio']=grid_grouplist.item_groupllist_target  /grid_grouplist.item_groupllist_count
grid_grouplist['ratio']=grid_grouplist['ratio'].fillna(0)

del gb
gc.collect()
 


 


0

In [ ]:
(grid1[s_items1_cols]>0).any(axis=1)

0            True
1            True
2            True
3            True
4            True
            ...  
11097207     True
11097208     True
11097209     True
11097210     True
11097211    False
Length: 11097212, dtype: bool

In [ ]:
grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False)

,shop_id,item_id,date_block_num,target,mean_price_month,min_price_month,max_price_month,std_price_month,item_category_id,cd_count,bd_count,dvd_count,lp_count,code_item_id,code_item_id1,text_feature_0,text_feature_1,text_feature_2,text_feature_3,text_feature_4,text_feature_5,text_feature_6,text_feature_7,text_feature_8,text_feature_9,text_feature_10,text_feature_11,text_feature_12,text_feature_13,text_feature_14,text_feature_15,text_feature_16,text_feature_17,text_feature_18,text_feature_19,text_feature_20,games_all,class1,class2,class4,shop_type,city_code,item_groupllist_target,item_groupllist_count,ratio
10499752,55,13834,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,11693,2613,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,949.0,163.0,5.822086
10619066,7,16443,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,13490,3046,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,3,4,949.0,163.0,5.822086
10451998,38,16494,32,1.0,169.0,169.0,169.0,NaN,40,0,0,0,0,13688,3071,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
10451997,38,15836,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,14248,3287,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
10451996,38,15833,32,0.0,NaN,NaN,NaN,NaN,40,0,0,0,0,14248,3287,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,14,949.0,163.0,5.822086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10551504,16,4418,32,0.0,NaN,NaN,NaN,NaN,55,0,0,0,0,3356,688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8,0.0,0.0,0.000000
10551505,16,4427,32,0.0,NaN,NaN,NaN,NaN,55,0,0,0,0,3353,688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,8,0.0,0.0,0.000000
10551643,16,21416,32,0.0,NaN,NaN,NaN,NaN,37,0,1,0,0,16210,3993,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,4,8,0.0,0.0,0.000000
10551657,16,21872,32,0.0,NaN,NaN,NaN,NaN,37,0,1,0,0,17536,4366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,4,8,0.0,0.0,0.000000


In [ ]:
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].item_groupllist_count.value_counts())
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False).shape)

1.0      47124
0.0      39270
2.0      20244
3.0      10080
4.0       8736
163.0     6846
5.0       6132
122.0     5124
29.0      4872
6.0       4032
93.0      3906
7.0       3822
11.0      3696
21.0      3528
28.0      3528
76.0      3192
74.0      3108
10.0      2940
17.0      2856
16.0      2730
8.0       2688
63.0      2646
58.0      2436
14.0      2352
54.0      2268
13.0      2184
24.0      2016
48.0      2016
9.0       1890
34.0      1428
15.0      1260
25.0      1050
12.0      1008
22.0       924
20.0       840
19.0       798
Name: item_groupllist_count, dtype: int64
(213570, 45)


In [ ]:
#just to have a look how good this aggregations are 
#print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False)[80000:])
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].item_groupllist_count.value_counts())
print(grid_grouplist.loc[grid_grouplist.date_block_num==32].sort_values('item_groupllist_count', axis=0, ascending=False).shape)

1.0      47124
0.0      39270
2.0      20244
3.0      10080
4.0       8736
163.0     6846
5.0       6132
122.0     5124
29.0      4872
6.0       4032
93.0      3906
7.0       3822
11.0      3696
21.0      3528
28.0      3528
76.0      3192
74.0      3108
10.0      2940
17.0      2856
16.0      2730
8.0       2688
63.0      2646
58.0      2436
14.0      2352
54.0      2268
13.0      2184
24.0      2016
48.0      2016
9.0       1890
34.0      1428
15.0      1260
25.0      1050
12.0      1008
22.0       924
20.0       840
19.0       798
Name: item_groupllist_count, dtype: int64
(213570, 45)


In [ ]:
grouplist_save_cols=[	'shop_id',	'item_id' ]+grouplist+['item_groupllist_target',	'item_groupllist_count',	'ratio']
grid_grouplist=grid_grouplist[grouplist_save_cols]
grid_grouplist.name="grid_grouplist"
print(grid_grouplist.head( ))

 


   shop_id  item_id  text_feature_0  text_feature_1  text_feature_2  \
0       59    22154               0               0               0   
1       59     2552             181              86               0   
2       59     2554             181               0               0   
3       59     2555             181               0               0   
4       59     2564             181               0               0   

   text_feature_3  text_feature_4  text_feature_5  text_feature_6  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               8               0               0              55   
4               0               0               0               0   

   text_feature_7  text_feature_8  text_feature_9  text_feature_10  \
0               0               0               0                0   
1               0 

In [ ]:
index_cols=[	'shop_id',	'item_id' ]+grouplist
lags=[1,2,3,4]
grid_grouplist=create_lags( grid_grouplist,index_cols, shift_range=lags)
grid_grouplist=grid_grouplist.fillna(0)
cols_diff=['item_groupllist_target','item_groupllist_count','ratio']  
grid_grouplist=difference(grid_grouplist,cols_diff)

index_cols =  ['shop_id', 'item_id', 'text_feature_0', 'text_feature_1', 'text_feature_2', 'text_feature_3', 'text_feature_4', 'text_feature_5', 'text_feature_6', 'text_feature_7', 'text_feature_8', 'text_feature_9', 'text_feature_10', 'text_feature_11', 'text_feature_12', 'text_feature_13', 'text_feature_14', 'text_feature_15', 'text_feature_16', 'text_feature_17', 'text_feature_18', 'text_feature_19', 'text_feature_20', 'date_block_num']
['item_groupllist_target', 'ratio', 'item_groupllist_count']



cols_to_rename (dropped) ['item_groupllist_target', 'ratio', 'item_groupllist_count']


In [ ]:
#We don't need to save all features in the grid_grouplist file 
is_nan_check(grid_grouplist)
grouplist.remove("date_block_num")
print(grouplist)
grid_grouplist=grid_grouplist.drop(grouplist, axis=1)
print(grid_grouplist)

shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
text_feature_0  is nan count  0  ; mean  0.0
text_feature_1  is nan count  0  ; mean  0.0
text_feature_2  is nan count  0  ; mean  0.0
text_feature_3  is nan count  0  ; mean  0.0
text_feature_4  is nan count  0  ; mean  0.0
text_feature_5  is nan count  0  ; mean  0.0
text_feature_6  is nan count  0  ; mean  0.0
text_feature_7  is nan count  0  ; mean  0.0
text_feature_8  is nan count  0  ; mean  0.0
text_feature_9  is nan count  0  ; mean  0.0
text_feature_10  is nan count  0  ; mean  0.0
text_feature_11  is nan count  0  ; mean  0.0
text_feature_12  is nan count  0  ; mean  0.0
text_feature_13  is nan count  0  ; mean  0.0
text_feature_14  is nan count  0  ; mean  0.0
text_feature_15  is nan count  0  ; mean  0.0
text_feature_16  is nan count  0  ; mean  0.0
text_feature_17  is nan count  0  ; mean  0.0
text_feature_18  is nan count  0  ; mean  0.0
text_feature_19  is nan count  0  ; mean  0.0
text_feature_2

In [ ]:
#grid_grouplist.name="grid_grouplist"
grid_grouplist.name="grid_grouplist_lvl_3"
save_csv(grid_grouplist)
del grid_grouplist
gc.collect()

original dataframe size 1100.6451110839844  MBs
downcasted  dataframe  size 465.6575469970703  MBs
fraction of original memory size:  0.4230769230769231
shop_id                           int16
item_id                           int16
date_block_num                    int16
item_groupllist_target_lag_1    float16
ratio_lag_1                     float16
item_groupllist_count_lag_1     float16
item_groupllist_target_lag_2    float16
ratio_lag_2                     float16
item_groupllist_count_lag_2     float16
item_groupllist_target_lag_3    float16
ratio_lag_3                     float16
item_groupllist_count_lag_3     float16
item_groupllist_target_lag_4    float16
ratio_lag_4                     float16
item_groupllist_count_lag_4     float16
item_groupllist_target_diff     float16
item_groupllist_count_diff      float16
ratio_diff                      float16
dtype: object
         shop_id  item_id  date_block_num  item_groupllist_target_lag_1  \
5548606       59     2754             

55

In [ ]:
print(sales.columns.values)
print(grid1.columns.values)
print(s_items1_cols)

['date_block_num' 'item_cnt_day' 'item_id' 'item_price' 'shop_id' 'month'
 'day_of_mnth' 'item_category_id' 'shop_type' 'city_code' 'games_all'
 'class1' 'class2' 'class4' 'cd_count' 'bd_count' 'dvd_count' 'lp_count'
 'code_item_id' 'code_item_id1' 'text_feature_0' 'text_feature_1'
 'text_feature_2' 'text_feature_3' 'text_feature_4' 'text_feature_5'
 'text_feature_6' 'text_feature_7' 'text_feature_8' 'text_feature_9'
 'text_feature_10' 'text_feature_11' 'text_feature_12' 'text_feature_13'
 'text_feature_14' 'text_feature_15' 'text_feature_16' 'text_feature_17'
 'text_feature_18' 'text_feature_19' 'text_feature_20']
['shop_id' 'item_id' 'date_block_num' 'target' 'mean_price_month'
 'min_price_month' 'max_price_month' 'std_price_month' 'item_category_id'
 'cd_count' 'bd_count' 'dvd_count' 'lp_count' 'code_item_id'
 'code_item_id1' 'text_feature_0' 'text_feature_1' 'text_feature_2'
 'text_feature_3' 'text_feature_4' 'text_feature_5' 'text_feature_6'
 'text_feature_7' 'text_feature_8' 'tex

In [ ]:
#NEW FILE BEGINS
#mean-encode text features: need to do something with zero values (when all features are zero?)(mean across all items or active items only? where the target is non-zero)
gc.collect()
counter=0
necessary_features=['date_block_num', 'shop_id','item_id','target']+s_items1_cols
grid_text_features=grid1[necessary_features]
for x in s_items1_cols:
  
  groupby_cols=['date_block_num', 'shop_id',x]
  feature='text_feature_{}_enc'.format(counter)
  gb=grid1.groupby(groupby_cols).agg(feature=("target","mean")) 
  grid_text_features = pd.merge(grid_text_features, gb, how='left', on=groupby_cols)
  grid_text_features.rename(columns={"feature": feature},inplace=True )
  counter+=1

#Downcast not needed!!  
  

In [ ]:
grid_text_features [1000:10010]
grid_text_features =grid_text_features.drop(s_items1_cols+['target'],axis=1)

In [ ]:
grid_text_features [1000:10010]

,date_block_num,shop_id,item_id,text_feature_0_enc,text_feature_1_enc,text_feature_2_enc,text_feature_3_enc,text_feature_4_enc,text_feature_5_enc,text_feature_6_enc,text_feature_7_enc,text_feature_8_enc,text_feature_9_enc,text_feature_10_enc,text_feature_11_enc,text_feature_12_enc,text_feature_13_enc,text_feature_14_enc,text_feature_15_enc,text_feature_16_enc,text_feature_17_enc,text_feature_18_enc,text_feature_19_enc,text_feature_20_enc
1000,0,59,8460,0.184082,0.164429,0.040985,0.204834,0.244629,0.002048,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1001,0,59,1958,0.184082,0.164429,0.790527,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.741211,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1002,0,59,1722,0.184082,0.399902,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1003,0,59,3152,0.184082,0.164429,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.239868,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
1004,0,59,3159,0.184082,0.164429,0.249756,0.204834,0.244629,0.250000,0.244995,0.233154,0.248169,0.230957,0.238037,0.220459,0.248047,0.246948,0.620605,0.234741,0.240356,0.239258,0.235474,0.241211,0.246826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10005,0,25,12652,0.861816,0.809570,0.953613,0.900879,0.942383,0.969238,0.939453,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10006,0,25,12653,0.861816,0.514160,0.643066,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10007,0,25,12654,0.861816,0.514160,0.643066,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707
10008,0,25,12658,0.391357,0.809570,0.953613,0.900879,0.942383,0.969238,0.695801,0.920898,0.959473,0.924805,0.930176,0.910645,0.954590,0.959473,0.938965,0.937988,0.936523,0.936523,0.937012,0.940918,0.949707


In [ ]:

lags=[1,2]
index_cols=['date_block_num', 'shop_id','item_id']
grid_text_features=create_lags( grid_text_features,index_cols, shift_range=lags) 

index_cols =  ['date_block_num', 'shop_id', 'item_id']
['text_feature_7_enc', 'text_feature_0_enc', 'text_feature_16_enc', 'text_feature_13_enc', 'text_feature_8_enc', 'text_feature_12_enc', 'text_feature_17_enc', 'text_feature_20_enc', 'text_feature_6_enc', 'text_feature_2_enc', 'text_feature_9_enc', 'text_feature_1_enc', 'text_feature_4_enc', 'text_feature_11_enc', 'text_feature_10_enc', 'text_feature_5_enc', 'text_feature_3_enc', 'text_feature_15_enc', 'text_feature_14_enc', 'text_feature_18_enc', 'text_feature_19_enc']



cols_to_rename (dropped) ['text_feature_7_enc', 'text_feature_0_enc', 'text_feature_16_enc', 'text_feature_13_enc', 'text_feature_8_enc', 'text_feature_12_enc', 'text_feature_17_enc', 'text_feature_20_enc', 'text_feature_6_enc', 'text_feature_2_enc', 'text_feature_9_enc', 'text_feature_1_enc', 'text_feature_4_enc', 'text_feature_11_enc', 'text_feature_10_enc', 'text_feature_5_enc', 'text_feature_3_enc', 'text_feature_15_enc', 'text_feature_14_enc', 'text_feature_18_enc', 'text_feature_19_enc']


In [ ]:
grid_text_features [1000000:10010000]

,date_block_num,shop_id,item_id,text_feature_7_enc_lag_1,text_feature_0_enc_lag_1,text_feature_16_enc_lag_1,text_feature_13_enc_lag_1,text_feature_8_enc_lag_1,text_feature_12_enc_lag_1,text_feature_17_enc_lag_1,text_feature_20_enc_lag_1,text_feature_6_enc_lag_1,text_feature_2_enc_lag_1,text_feature_9_enc_lag_1,text_feature_1_enc_lag_1,text_feature_4_enc_lag_1,text_feature_11_enc_lag_1,text_feature_10_enc_lag_1,text_feature_5_enc_lag_1,text_feature_3_enc_lag_1,text_feature_15_enc_lag_1,text_feature_14_enc_lag_1,text_feature_18_enc_lag_1,text_feature_19_enc_lag_1,text_feature_7_enc_lag_2,text_feature_0_enc_lag_2,text_feature_16_enc_lag_2,text_feature_13_enc_lag_2,text_feature_8_enc_lag_2,text_feature_12_enc_lag_2,text_feature_17_enc_lag_2,text_feature_20_enc_lag_2,text_feature_6_enc_lag_2,text_feature_2_enc_lag_2,text_feature_9_enc_lag_2,text_feature_1_enc_lag_2,text_feature_4_enc_lag_2,text_feature_11_enc_lag_2,text_feature_10_enc_lag_2,text_feature_5_enc_lag_2,text_feature_3_enc_lag_2,text_feature_15_enc_lag_2,text_feature_14_enc_lag_2,text_feature_18_enc_lag_2,text_feature_19_enc_lag_2
1000000,2,8,9546,0.134399,0.118896,0.068970,0.142456,0.141113,0.142334,0.138550,0.142334,0.139282,0.143188,0.136108,0.113159,0.138916,0.134033,0.138672,0.140625,0.129028,0.137939,0.139526,0.138916,0.139771,0.171997,0.138794,0.000000,0.182129,0.178345,0.181030,0.175903,0.180054,0.178345,0.172241,0.171631,0.132324,0.178955,0.161743,0.173828,0.176514,0.154785,0.172729,0.177002,0.172852,0.177368
1000001,2,8,9539,0.134399,0.118896,0.139160,0.172729,0.141113,0.142334,0.138550,0.142334,0.139282,0.143188,0.136108,0.113159,0.138916,0.134033,0.138672,0.140625,0.129028,0.241943,0.139526,0.138916,0.139771,0.171997,0.138794,0.176636,0.103760,0.178345,0.181030,0.175903,0.180054,0.178345,0.172241,0.171631,0.132324,0.178955,0.161743,0.173828,0.176514,0.154785,0.290283,0.177002,0.172852,0.177368
1000002,2,8,6882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000003,2,8,6848,0.134399,0.118896,0.139160,0.142456,0.141113,0.142334,0.138550,0.142334,0.139282,0.143188,0.136108,0.113159,0.138916,0.134033,0.138672,0.140625,0.129028,0.137939,0.139526,0.138916,0.139771,0.171997,0.138794,0.176636,0.182129,0.178345,0.181030,0.175903,0.180054,0.178345,0.172241,0.171631,0.132324,0.178955,0.161743,0.173828,0.176514,0.154785,0.172729,0.177002,0.172852,0.177368
1000004,2,8,6818,0.134399,0.118896,0.139160,0.142456,0.141113,0.142334,0.138550,0.142334,0.139282,0.143188,0.136108,0.113159,0.138916,0.203735,0.138672,0.140625,0.129028,0.137939,0.139526,0.138916,0.139771,0.171997,0.138794,0.176636,0.182129,0.178345,0.181030,0.175903,0.180054,0.178345,0.172241,0.171631,0.132324,0.178955,0.500000,0.173828,0.176514,0.154785,0.172729,0.177002,0.172852,0.177368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10009995,30,58,15740,0.296875,0.280518,0.297119,0.301270,0.285645,0.299805,0.297607,0.294678,0.275146,0.311279,0.282715,0.240967,0.298340,0.284180,0.297119,0.314941,0.302246,0.283691,0.279053,0.293213,0.294922,0.260742,0.248291,0.259766,0.257812,0.246338,0.259277,0.259033,0.254395,0.245361,0.275391,0.247681,0.211792,0.256104,0.247803,0.258789,0.272949,0.244263,0.247803,0.249512,0.255371,0.253662
10009996,30,58,15591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108826,0.248291,0.259766,0.257812,0.246338,0.259277,0.259033,0.254395,0.245361,0.275391,0.247681,0.211792,0.256104,0.247803,0.258789,0.272949,0.244263,0.247803,0.249512,0.255371,0.253662
10009997,30,58,16237,0.296875,0.280518,0.297119,0.301270,0.410156,0.299805,0.297607,0.294678,0.275146,0.281982,0.282715,0.240967,0.298340,0.284180,0.297119,0.314941,0.323975,0.283691,0.279053,0.293213,0.294922,0.260742,0.248291,0.259766,0.257812,0.508789

In [ ]:
is_nan_check(grid_text_features[grid_text_features.date_block_num>1])

date_block_num  is nan count  0  ; mean  0.0
shop_id  is nan count  0  ; mean  0.0
item_id  is nan count  0  ; mean  0.0
text_feature_7_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_0_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_16_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_13_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_8_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_12_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_17_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_20_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_6_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_2_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_9_enc_lag_1  is nan count  2055720  ; mean  0.19849929159124163
text_feature_1_enc_lag_1  i

In [ ]:

grid_text_features.name="grid_text_features_enc"
save_csv(grid_text_features)
del grid_text_features
gc.collect()

original dataframe size 1037.146354675293  MBs
downcasted  dataframe  size 1037.146354675293  MBs
fraction of original memory size:  1.0
date_block_num                 int16
shop_id                        int16
item_id                        int16
text_feature_7_enc_lag_1     float16
text_feature_0_enc_lag_1     float16
text_feature_16_enc_lag_1    float16
text_feature_13_enc_lag_1    float16
text_feature_8_enc_lag_1     float16
text_feature_12_enc_lag_1    float16
text_feature_17_enc_lag_1    float16
text_feature_20_enc_lag_1    float16
text_feature_6_enc_lag_1     float16
text_feature_2_enc_lag_1     float16
text_feature_9_enc_lag_1     float16
text_feature_1_enc_lag_1     float16
text_feature_4_enc_lag_1     float16
text_feature_11_enc_lag_1    float16
text_feature_10_enc_lag_1    float16
text_feature_5_enc_lag_1     float16
text_feature_3_enc_lag_1     float16
text_feature_15_enc_lag_1    float16
text_feature_14_enc_lag_1    float16
text_feature_18_enc_lag_1    float16
text_feature

175

In [ ]:
 
index_cols=['item_id', 'city_code', 'date_block_num']

grid_item_city=grid_df1( sales,index_cols) 




gb = grid1.loc[grid1.target>0].groupby(index_cols ).agg(target_item_city_code=('target' , 'sum' ),item_active_city_code_count=('shop_id', 'nunique')).fillna(0)
grid_item_city = pd.merge(grid_item_city, gb, how='left', on=index_cols) 

grid_item_city.target_item_city_code=grid_item_city.target_item_city_code.fillna(0)
grid_item_city.item_active_city_code_count=grid_item_city.item_active_city_code_count.fillna(0)
 
 
grid_item_city['mean_item_per_active_city_code']=grid_item_city.target_item_city_code/grid_item_city.item_active_city_code_count 
grid_item_city['mean_item_per_active_city_code']=grid_item_city['mean_item_per_active_city_code'].fillna(0)  

 

In [ ]:
grid_item_city=create_lags( grid_item_city,index_cols, shift_range=[1,2,3,4,12])

In [ ]:
is_nan_check(grid_item_city)

In [ ]:
grid_item_city.loc[grid_item_city.date_block_num==25].head(20)
grid_item_city=grid_item_city.fillna(0)

In [ ]:
cols_diff=['target_item_city_code', 'item_active_city_code_count', 'mean_item_per_active_city_code']  
grid_item_city=difference(grid_item_city,cols_diff)
grid_item_city=grid_item_city.fillna(0)

In [ ]:
grid_item_city.name="grid_item_city"
save_csv(grid_item_city)
print(grid_item_city.head( ))

del grid_item_city,gb
gc.collect()



In [ ]:
  
index_cols=['shop_id','code_item_id1','date_block_num']

grid_shop_code1_month=grid_df1( sales,index_cols) 
 
gb = grid1.loc[grid1.target>0].groupby(index_cols ).agg(target_shop_code1=('target' , 'sum' ),shop_code1_count=('shop_id', 'nunique')).fillna(0)
grid_shop_code1_month = pd.merge(grid_shop_code1_month, gb, how='left', on=index_cols) 


grid_shop_code1_month.target_shop_code1=grid_shop_code1_month.target_shop_code1.fillna(0)
grid_shop_code1_month.shop_code1_count=grid_shop_code1_month.shop_code1_count.fillna(0)
 

 
grid_shop_code1_month['mean_target_shop_code1']=grid_shop_code1_month.target_shop_code1/grid_shop_code1_month.shop_code1_count
grid_shop_code1_month['mean_target_shop_code1']=grid_shop_code1_month['mean_target_shop_code1'].fillna(0)  

grid_shop_code1_month=grid_shop_code1_month.fillna(0)

grid_shop_code1_month=create_lags( grid_shop_code1_month,index_cols, shift_range=[1,2,3,4,12])
grid_shop_code1_month.name="grid_shop_code1_month"

grid_shop_code1_month=grid_shop_code1_month.fillna(0)







In [ ]:
grid_shop_code1_month.name="grid_shop_code1_month"
save_csv(grid_shop_code1_month)

del grid_shop_code1_month,gb
gc.collect() 

In [ ]:

grid1['rev_per_item']=grid1['target']*grid1['mean_price_month']

index_cols=[  'shop_id','date_block_num'] 
gb=grid1.groupby (index_cols).agg(shop_rev=('rev_per_item', 'sum')).fillna(0) 

grid1=pd.merge(grid1,gb,  how='left', on=index_cols) 

grid1.rev_per_item=grid1.rev_per_item.fillna(0)
grid1.shop_rev=grid1.shop_rev.fillna(0)

In [ ]:
print(grid1.columns.values)
to_drop=grouplist+['cd_count', 'bd_count', 'dvd_count' ,'lp_count','code_item_id', 'code_item_id1']
grid1=grid1.drop(to_drop, axis=1)
print(grid1.columns.values)
is_nan_check(grid1)

In [ ]:
#grid1=grid1.drop(['cd_count', 'bd_count', 'dvd_count' ,'lp_count','code_item_id', 'code_item_id1'], axis=1)

In [ ]:
print(grid1.columns.values)
 
 

In [ ]:
grid1=grid1.drop( 'item_category_id', axis=1)
index_cols=['shop_id' ,'item_id' ,'date_block_num']

shift_range=[1,2,3,4,12]
grid1=create_lags( grid1,index_cols, shift_range=shift_range)

for x in ['target','rev_per_item', 'shop_rev']:
  for y in shift_range:
    col='{}_lag_{}'.format(x,y)
    grid1[col]=grid1[col].fillna(0)

cols_diff=['target','mean_price_month','std_price_month','rev_per_item', 'shop_rev']  
grid1=difference(grid1,cols_diff)

for x in ['target','rev_per_item', 'shop_rev']:
  col='{}_diff'.format(x)
  grid1[col]=grid1[col].fillna(0)




 


In [ ]:

is_nan_check(grid1)

In [ ]:
print(history_shop)
grid1=pd.merge(grid1,history_shop, how='left', on=['item_id','shop_id'])


In [ ]:
#we don't know in which shops the item will be sold, so value 0 means 'no history' 
grid1['shop_history']=(grid1[ 'date_block_num']-grid1['item_shop_start']).clip(0,35)

grid1['shop_history']=grid1['shop_history'].fillna(0)
grid1=grid1.drop(['item_shop_start','item_shop_end'],axis=1)
is_nan_check(grid1,cols=['shop_history'])

In [ ]:
print(grid1)
grid1.loc[ grid1.item_id==15757].groupby('date_block_num').agg(t=('target','sum'))

In [ ]:
cols=list(grid1.columns.values)

for col in cols:
     
    grid1[col].replace(np.inf, 0, inplace=True) 
print(grid1.shop_rev_lag_1)

In [ ]:
#duct tape
#for col in cols:
     
#    print(grid1.loc[grid1[col]==np.inf] )

In [ ]:
grid1.name="grid1"
save_csv(grid1)

#del grid1,gb
gc.collect() 

In [ ]:
#grid1 = pd.read_csv(path+'grid1.csv',usecols=['shop_id','date_block_num'] , encoding='utf8')

In [ ]:
grid1.loc[grid1['date_block_num']==33].shape 

print(grid1.loc[grid1['date_block_num']==33].shop_id.value_counts())
print(grid1.loc[grid1['date_block_num']==33].shop_id.nunique())

In [ ]:
gc.collect()

###Clean until here!!
###Clean until here!!
###Clean until here!!
###Clean until here!!
